In [2]:
# crawl link buy
from bs4 import BeautifulSoup
import requests

# Khởi tạo một danh sách để lưu trữ các URL
all_urls = []

# Sử dụng vòng lặp để duyệt qua các trang
for i in range(1, 5):  # Bạn có thể thay đổi số lượng trang tùy ý
    url = f'https://alonhadat.com.vn/nha-dat/can-ban/trang--{i}.html'
    
    # Gửi yêu cầu GET đến trang web
    response = requests.get(url)
    
    # Kiểm tra xem yêu cầu có thành công không
    if response.status_code != 200:
        print(f"Không thể truy cập trang {url}")
        continue
    
    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Tìm tất cả các thẻ <a> trong <div class="content-item">
    links = soup.select('div.content-item div.ct_title a')

    # Lặp qua các liên kết và thêm vào danh sách
    for link in links:
        href = link.get('href')
        if href and href.startswith('/'):  # Đảm bảo liên kết là đường dẫn tương đối
            full_link = f"https://alonhadat.com.vn{href}"
            all_urls.append(full_link)

# Ghi các URL vào một file văn bản
file_path = "buy_urls.txt"
with open(file_path, "w") as file:
    for url in all_urls:
        file.write(url + "\n")

print(f"Đã lưu các URL vào file {file_path}")


Đã lưu các URL vào file buy_urls.txt


In [3]:
# crawl link - rent 
from bs4 import BeautifulSoup
import requests

# Khởi tạo một danh sách để lưu trữ các URL
all_urls = []

# Sử dụng vòng lặp để duyệt qua các trang
for i in range(1, 5):  # Bạn có thể thay đổi số lượng trang tùy ý
    url = f'https://alonhadat.com.vn/nha-dat/cho-thue/trang--{i}.html'
    
    # Gửi yêu cầu GET đến trang web
    response = requests.get(url)
    
    # Kiểm tra xem yêu cầu có thành công không
    if response.status_code != 200:
        print(f"Không thể truy cập trang {url}")
        continue
    
    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Tìm tất cả các thẻ <a> trong <div class="content-item">
    links = soup.select('div.content-item div.ct_title a')

    # Lặp qua các liên kết và thêm vào danh sách
    for link in links:
        href = link.get('href')
        if href and href.startswith('/'):  # Đảm bảo liên kết là đường dẫn tương đối
            full_link = f"https://alonhadat.com.vn{href}"
            all_urls.append(full_link)

# Ghi các URL vào một file văn bản
file_path = "rent_urls.txt"
with open(file_path, "w") as file:
    for url in all_urls:
        file.write(url + "\n")

print(f"Đã lưu các URL vào file {file_path}")


Đã lưu các URL vào file rent_urls.txt


In [4]:
from bs4 import BeautifulSoup
import requests
import json
import re

def extract_text(soup, selector):
    element = soup.select_one(selector)
    return element.text.strip() if element else None

def extract_text_from_td(td_elements, label):
    for i in range(len(td_elements) - 1):
        if td_elements[i].text.strip() == label:
            return td_elements[i + 1].text.strip()
    return None

def replace_underscore_with_null(value):
    return None if value == "_" or value == "---" else value

def process_address(address):
    parts = address.split(',')
    if len(parts) >= 3:
        city = parts[-1].strip()
        district = parts[-2].strip()
        full_address = ', '.join(parts[:-2]).strip()
        combined_address = f"{full_address}, {district}, {city}"
        return combined_address, city, district
    return address, None, None

def extract_phone_number(soup):
    try:
        phone_element = soup.select_one("div.fone a")
        if phone_element and phone_element.has_attr('href') and phone_element['href'].startswith('tel:'):
            phone_number = phone_element['href'].replace('tel:', '').strip()
            return phone_number
        else:
            return None
    except Exception as e:
        print(f"Lỗi khi trích xuất số điện thoại: {e}")
        return None

def crawl_data_from_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url.strip(), headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            title = extract_text(soup, "div.title h1")

            detail = soup.select_one("div.detail.text-content")
            if detail:
                # Lấy nội dung văn bản và loại bỏ khoảng trắng đầu cuối
                detail_text = detail.get_text(strip=True)
                
                # Loại bỏ các ký tự không mong muốn như \r
                detail_text = re.sub(r'[^\w\s,.()\–]', '', detail_text)
                detail_text = detail_text.replace('\r', '')
            else:
                detail_text = None

            price = extract_text(soup, "span.price .value")
            area = extract_text(soup, "span.square .value")

            address = extract_text(soup, "div.address .value")
            combined_address, city, district = process_address(address)

            more_info = soup.select("div.moreinfor1 .infor td")

            property_id = extract_text_from_td(more_info, "Mã tin")
            direction = extract_text_from_td(more_info, "Hướng")
            type_of_listing = extract_text_from_td(more_info, "Loại tin")
            road_width = extract_text_from_td(more_info, "Đường trước nhà")
            bedrooms = extract_text_from_td(more_info, "Số phòng ngủ")
            floors = extract_text_from_td(more_info, "Số lầu")
            legal_status = extract_text_from_td(more_info, "Pháp lý")
            property_type = extract_text_from_td(more_info, "Loại BDS")
            width = extract_text_from_td(more_info, "Chiều ngang")
            length = extract_text_from_td(more_info, "Chiều dài")

            image = soup.select_one("div.images img")
            image_link = image['src'] if image else None
            full_image_link = "https://alonhadat.com.vn/" + image_link if image_link else None
            
            # Thay thế giá trị "_" bằng null
            property_id = replace_underscore_with_null(property_id)
            direction = replace_underscore_with_null(direction)
            type_of_listing = replace_underscore_with_null(type_of_listing)
            road_width = replace_underscore_with_null(road_width)
            bedrooms = replace_underscore_with_null(bedrooms)
            floors = replace_underscore_with_null(floors)
            legal_status = replace_underscore_with_null(legal_status)
            property_type = replace_underscore_with_null(property_type)
            width = replace_underscore_with_null(width)
            length = replace_underscore_with_null(length)

            # Trích xuất số điện thoại
            phone_number = extract_phone_number(soup)

            if full_image_link is None:
                return None
            
            return {
                "title": title or None,
                "description": detail_text or None,
                "price": price or None,
                "area": area or None,
                "address": combined_address or None,
                "province_city": city or None,
                "district": district or None,
                "direction": direction,
                "type": 'Mua Bán',
                "bedroom": bedrooms,
                "toilet": bedrooms, 
                "floor": floors,
                "law": legal_status,
                "type_estate": property_type,
                "width": width,
                "length": length,
                'image_links': full_image_link,                
                "phonenumber": phone_number
            }
        else:
            print(f"Lỗi khi truy cập vào URL: {url}")
            return None
    except requests.exceptions.Timeout:
        print(f"Timeout khi truy cập vào URL: {url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Lỗi kết nối khi truy cập vào URL: {url}, Lỗi: {e}")
        return None
    except Exception as e:
        print(f"Lỗi khi trích xuất thông tin từ URL: {url}, Lỗi: {e}")
        return None

data_list = []

with open("buy_urls.txt", "r", encoding="utf-8") as file:
    urls = file.readlines()

for url in urls:
    url = url.strip()
    item = crawl_data_from_url(url)
    if item:
        data_list.append(item)

with open("buy_data.json", "w", encoding="utf-8") as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, indent=4)

print("Dữ liệu đã được lưu vào tệp buy_data.json")


Dữ liệu đã được lưu vào tệp data.json


In [ ]:

from bs4 import BeautifulSoup
import requests
import json

def extract_text(soup, selector):
    element = soup.select_one(selector)
    return element.text.strip() if element else None

def extract_text_from_td(td_elements, label):
    for i in range(len(td_elements) - 1):
        if td_elements[i].text.strip() == label:
            return td_elements[i + 1].text.strip()
    return None

def replace_underscore_with_null(value):
    return None if value == "_" or value == "---" else value

def process_address(address):
    parts = address.split(',')
    if len(parts) >= 3:
        city = parts[-1].strip()
        district = parts[-2].strip()
        full_address = ', '.join(parts[:-2]).strip()
        combined_address = f"{full_address}, {district}, {city}"
        return combined_address, city, district
    return address, None, None

def extract_phone_number(soup):
    try:
        phone_element = soup.select_one("div.fone a")
        if phone_element and phone_element.has_attr('href') and phone_element['href'].startswith('tel:'):
            phone_number = phone_element['href'].replace('tel:', '').strip()
            return phone_number
        else:
            return None
    except Exception as e:
        print(f"Lỗi khi trích xuất số điện thoại: {e}")
        return None
    
def crawl_data_from_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url.strip(), headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            title = extract_text(soup, "div.title h1")
            title = re.sub(r'[^\w\s,.()\–]', '', title)

            detail = soup.select_one("div.detail.text-content")
            if detail:
                # Lấy nội dung văn bản và loại bỏ khoảng trắng đầu cuối
                detail_text = detail.get_text(strip=True)
                
                # Loại bỏ các ký tự không mong muốn như \r
                detail_text = re.sub(r'[^\w\s,.()\–]', '', detail_text)
                detail_text = detail_text.replace('\r', '')
            else:
                detail_text = None

            price = extract_text(soup, "span.price .value")
            area = extract_text(soup, "span.square .value")

            address = extract_text(soup, "div.address .value")
            combined_address, city, district = process_address(address)

            more_info = soup.select("div.moreinfor1 .infor td")

            property_id = extract_text_from_td(more_info, "Mã tin")
            direction = extract_text_from_td(more_info, "Hướng")
            type_of_listing = extract_text_from_td(more_info, "Loại tin")
            road_width = extract_text_from_td(more_info, "Đường trước nhà")
            bedrooms = extract_text_from_td(more_info, "Số phòng ngủ")
            floors = extract_text_from_td(more_info, "Số lầu")
            legal_status = extract_text_from_td(more_info, "Pháp lý")
            property_type = extract_text_from_td(more_info, "Loại BDS")
            width = extract_text_from_td(more_info, "Chiều ngang")
            length = extract_text_from_td(more_info, "Chiều dài")

            image = soup.select_one("div.images img")
            image_link = image['src'] if image else None
            full_image_link = "https://alonhadat.com.vn/" + image_link if image_link else None
            
            
            # Thay thế giá trị "_" bằng null
            property_id = replace_underscore_with_null(property_id)
            direction = replace_underscore_with_null(direction)
            type_of_listing = replace_underscore_with_null(type_of_listing)
            road_width = replace_underscore_with_null(road_width)
            bedrooms = replace_underscore_with_null(bedrooms)
            floors = replace_underscore_with_null(floors)
            legal_status = replace_underscore_with_null(legal_status)
            property_type = replace_underscore_with_null(property_type)
            width = replace_underscore_with_null(width)
            length = replace_underscore_with_null(length)

            if full_image_link is None:
                return None
            
            phone_number = extract_phone_number(soup)
            
            return {
                "title": title or None,
                "description": detail_text or None,
                "price": price or None,
                "area": area or None,
                "address": combined_address or None,
                "province_city": city or None,
                "district": district or None,
                "direction": direction,
                "type": 'Cho Thuê',
                "bedroom": bedrooms,
                "toilet":bedrooms,
                "floor": floors,
                "law": legal_status,
                "type_estate": property_type,
                "width": width,
                'image_links': full_image_link,
                'phonenumber': phone_number
            }
        else:
            print(f"Lỗi khi truy cập vào URL: {url}")
            return None
    except requests.exceptions.Timeout:
        print(f"Timeout khi truy cập vào URL: {url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Lỗi kết nối khi truy cập vào URL: {url}, Lỗi: {e}")
        return None
    except Exception as e:
        print(f"Lỗi khi trích xuất thông tin từ URL: {url}, Lỗi: {e}")
        return None

data_list = []

with open("rent_urls.txt", "r", encoding="utf-8") as file:
    urls = file.readlines()

for url in urls:
    url = url.strip()
    item = crawl_data_from_url(url)
    if item:
        data_list.append(item)

with open("rent_data.json", "w", encoding="utf-8") as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, indent=4)

print("Dữ liệu đã được lưu vào tệp rent_data.json")
